# First Module
# Model Input Module

# Loading packages

In [ ]:
import sys
sys.path.append('Model_input')  # Add the directory containing your .py file to the system path
from reading_file import load_data
from merging_file import (merging_data, assign_fuel_costs, cluster_plants, assign_em_rates, long_wide, transmission_func,
                          ffill_ren_cost, ffill_ren_cap, cluster_and_aggregate, long_wide_load, storage_object, solar_object,
                          wind_object, gen_object, load_object, trans_object, plant_capacity, trans_index, renewable_transmission_cost,adjust_coal_generation_cost,
                          convert_keys_to_string, merge_dictionaries_and_format, adjust_nuclear_generation_cost, concat_filtered_plants)
import json
import numpy as np
import pickle
import warnings
warnings.filterwarnings("ignore")

# Loading Input Data, adjusting fuel costs, fixing emission data, aggregating power plants

In [2]:

(Plant_old, Plant, Transmission, Parsed, Input, NEEDS, Wind_generation_profile, Load, Wind_onshore_capacity,
 Wind_capital_cost, Solar_regional_capacity, Solar_generation_profile, Solar_capital_cost_photov,
 Solar_capacity_factor, Regional_Cost, Unit_Cost) = load_data()
# Merging power plants data
Plant_short = merging_data(Plant, Parsed)
# Assigning fuel cost
Plant_short_fixed_fuelC = assign_fuel_costs(Plant_short)
Plant_short_fixed_fuelC_coal = adjust_coal_generation_cost(Plant_short_fixed_fuelC)
Plant_short_fixed_fuelC_coal_Nuc = adjust_nuclear_generation_cost(Plant_short_fixed_fuelC)
# Replacing missing fuel cost
Plant_short_fixed_Em = assign_em_rates(Plant_short_fixed_fuelC_coal_Nuc, Plant_old)
# Aggregation of power plants
Plants_community, all_regions_clusters = cluster_plants(Plant_short_fixed_Em, 2000, 2000, 10, 4, 1, 1, 1)
# Aggregating the power plants
Plants_ungroup,  Plants_group = cluster_and_aggregate(Plants_community)
Plants_ungroup_extended = concat_filtered_plants(Plants_ungroup, Plant_short)

Time taken to cluster plants in WECC_CO: 0.1053168773651123 seconds
Time taken to cluster plants in WECC_WY: 0.012945890426635742 seconds
Time taken to cluster plants in MIS_INKY: 0.11491703987121582 seconds
Time taken to cluster plants in SPP_N: 0.6565308570861816 seconds
Time taken to cluster plants in S_C_TVA: 0.1839618682861328 seconds
Time taken to cluster plants in WEC_SDGE: 0.0150299072265625 seconds
Time taken to cluster plants in S_SOU: 0.37580394744873047 seconds
Time taken to cluster plants in FRCC: 0.4009232521057129 seconds
Time taken to cluster plants in PJM_EMAC: 0.39171409606933594 seconds
Time taken to cluster plants in MIS_MNWI: 0.431229829788208 seconds
Time taken to cluster plants in NY_Z_B: 0.013206720352172852 seconds
Time taken to cluster plants in MIS_LMI: 0.3544943332672119 seconds
Time taken to cluster plants in WEC_CALN: 0.4296538829803467 seconds
Time taken to cluster plants in S_VACA: 0.46800923347473145 seconds
Time taken to cluster plants in ERC_REST: 0.8

# Creat a wide version of the input datas

In [3]:
Wind_generation_profile_wide = long_wide(Wind_generation_profile)
Solar_generation_profile_wide = long_wide(Solar_generation_profile)
Transmission_Capacity, Transmission_Energy, Transmission_Cost = transmission_func(Transmission)
Wind_onshore_capacity, Solar_regional_capacity = ffill_ren_cap(Wind_onshore_capacity, Solar_regional_capacity)
Wind_capital_cost, Solar_capital_cost_photov = ffill_ren_cost(Wind_capital_cost, Solar_capital_cost_photov)
Load_wide = long_wide_load(Load)
Transmission_index = trans_index(Transmission_Capacity)
Plant_capacity_dic = plant_capacity(Plant_short)
Wind_trans_capital_cost_final, Solar_trans_capital_cost_photov_final, Wind_capital_cost_copy, Solar_capital_cost_photov_copy = renewable_transmission_cost(Unit_Cost, Regional_Cost, Wind_capital_cost, Solar_capital_cost_photov)

# Making object-oriented data dictionaries

In [4]:
Region = Load['Region'].unique()
links = trans_object(Transmission_Capacity, Transmission_Cost)
load_oo = load_object(Load_wide)
generator_oo = gen_object(Plants_group)
storage_oo = storage_object(Plants_group)
solar_oo = solar_object(Solar_generation_profile_wide, Solar_capital_cost_photov, Solar_regional_capacity, Solar_capital_cost_photov_copy, Plants_group, Region)
wind_oo = wind_object(Wind_generation_profile_wide, Wind_capital_cost, Wind_onshore_capacity, Wind_capital_cost_copy,  Plants_group, Region)


# Creating the sets

In [5]:
sets = {
    'region': list(Transmission_Capacity.index.unique()),
    'gen_type': list(Plants_ungroup[~Plants_ungroup["gen_type"].isin(["Solar PV", "Onshore Wind", "Energy Storage"])]["gen_type"].unique()),
    'solar_rc': list(Solar_regional_capacity["New Resource Class"].unique()),
    'wind_rc': list(Wind_onshore_capacity["New Resource Class"].unique()),
    'cost_class': list([1, 2, 3, 4, 5, 6])
}
sorted_sets = {key: sorted(value) for key, value in sets.items()}

# Creating nodes and links

In [6]:
# Combine all the dictionaries into one list
all_dicts = load_oo + generator_oo + storage_oo + solar_oo + wind_oo
# all_dicts = load_oo + generator_oo + storage_oo
# Merge all dictionaries in the list
nodes = merge_dictionaries_and_format(all_dicts)
# Create a dictionary to hold all objects
all_objects = {'nodes': nodes, 'links': links}
# Convert all nested keys to strings
all_objects_str_keys = convert_keys_to_string(all_objects)

# Creating nodes and links, saving output as JSON and pickle files

In [7]:
# Combine all the dictionaries into one list
all_dicts = load_oo + generator_oo + storage_oo + solar_oo + wind_oo
# all_dicts = load_oo + generator_oo + storage_oo
# Merge all dictionaries in the list
nodes = merge_dictionaries_and_format(all_dicts)
# Create a dictionary to hold all objects
all_objects = {'nodes': nodes, 'links': links}
# Convert all nested keys to strings
all_objects_str_keys = convert_keys_to_string(all_objects)

# Define the file path for saving the JSON file
input_file = 'all_input_objects.json'
# Save all objects as JSON
with open(input_file, 'w') as f:
    json.dump(all_objects_str_keys, f)

sorted_sets_str = {key: [str(item) for item in value] if isinstance(value, list) else int(value) if isinstance(value, (np.int64, int)) else str(value) for key, value in sorted_sets.items()}

input_sets_sorted = 'all_input_sets_sorted.json'

# Make sure to use sorted_sets_str for JSON serialization
with open(input_sets_sorted, 'w') as f:
    json.dump(sorted_sets_str, f)

# Specify the path to save the Power Plants pickle file
pickle_file_path1 = 'Plants_group.pickle'
pickle_file_path2 = 'Plants_ungroup_extended.pickle'

# Save the dictionary as a pickle file
with open(pickle_file_path1, 'wb') as f:
    pickle.dump(Plants_group, f)
with open(pickle_file_path2, 'wb') as f:
    pickle.dump(Plants_ungroup_extended, f)
